In [27]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd

import os
from time import time
from time import sleep

In [21]:
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'10',  # top 10
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': 'be418cc8-d60e-4d91-85a8-2bfae177f020',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  # print(data)  # Commented out so I don't get a huge wall of data
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [22]:
type(data)

dict

json normalize

In [23]:
pd.set_option('display.max_columns', None)

In [32]:
df = pd.json_normalize(data['data'])

Add Timestamp column

In [33]:
df["timestamp"] = pd.Timestamp("now")  # Take timestamp and date as of right now

In [34]:
df

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,platform,cmc_rank,self_reported_circulating_supply,self_reported_market_cap,tvl_ratio,last_updated,quote.USD.price,quote.USD.volume_24h,quote.USD.volume_change_24h,quote.USD.percent_change_1h,quote.USD.percent_change_24h,quote.USD.percent_change_7d,quote.USD.percent_change_30d,quote.USD.percent_change_60d,quote.USD.percent_change_90d,quote.USD.market_cap,quote.USD.market_cap_dominance,quote.USD.fully_diluted_market_cap,quote.USD.tvl,quote.USD.last_updated,platform.id,platform.name,platform.symbol,platform.slug,platform.token_address,timestamp
0,1,Bitcoin,BTC,bitcoin,9745,2013-04-28T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",2.100000e+07,1.915168e+07,1.915168e+07,NaN,1,None,None,None,2022-09-16T00:08:00.000Z,19677.816577,3.629979e+10,-4.0526,-0.150769,-2.769594,1.831606,-17.574662,-5.598777,-4.034745,3.768633e+11,39.2255,4.132341e+11,None,2022-09-16T00:08:00.000Z,NaN,NaN,NaN,NaN,NaN,2022-09-15 20:13:21.320118
1,1027,Ethereum,ETH,ethereum,6091,2015-08-07T00:00:00.000Z,"[mineable, pos, smart-contracts, ethereum-ecos...",NaN,1.223830e+08,1.223830e+08,NaN,2,None,None,None,2022-09-16T00:08:00.000Z,1469.051888,2.682196e+10,49.3984,-0.208986,-10.348802,-10.126020,-21.805986,9.208606,35.133549,1.797870e+11,18.7098,1.797870e+11,None,2022-09-16T00:08:00.000Z,NaN,NaN,NaN,NaN,NaN,2022-09-15 20:13:21.320118
2,825,Tether,USDT,tether,39753,2015-02-25T00:00:00.000Z,"[payments, stablecoin, asset-backed-stablecoin...",NaN,6.788344e+10,6.915678e+10,NaN,3,None,None,None,2022-09-16T00:08:00.000Z,1.000054,6.044253e+10,9.6747,0.005628,-0.008061,-0.003668,-0.005704,0.033709,0.111695,6.788713e+10,7.0660,6.916055e+10,None,2022-09-16T00:08:00.000Z,1027.0,Ethereum,ETH,ethereum,0xdac17f958d2ee523a2206206994597c13d831ec7,2022-09-15 20:13:21.320118
3,3408,USD Coin,USDC,usd-coin,6229,2018-10-08T00:00:00.000Z,"[medium-of-exchange, stablecoin, asset-backed-...",NaN,5.035768e+10,5.035768e+10,NaN,4,None,None,None,2022-09-16T00:08:00.000Z,0.999964,7.128281e+09,23.2724,0.013297,-0.010201,-0.009295,-0.020866,-0.032737,-0.032982,5.035586e+10,5.2404,5.035586e+10,None,2022-09-16T00:08:00.000Z,1027.0,Ethereum,ETH,ethereum,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,2022-09-15 20:13:21.320118
4,1839,BNB,BNB,bnb,1103,2017-07-25T00:00:00.000Z,"[marketplace, centralized-exchange, payments, ...",2.000000e+08,1.613373e+08,1.613373e+08,NaN,5,None,None,None,2022-09-16T00:08:00.000Z,270.133149,1.078401e+09,12.7498,0.184893,-3.443865,-3.679453,-14.643333,8.664235,24.925458,4.358254e+10,4.5374,5.402663e+10,None,2022-09-16T00:08:00.000Z,NaN,NaN,NaN,NaN,NaN,2022-09-15 20:13:21.320118
5,4687,Binance USD,BUSD,binance-usd,5095,2019-09-20T00:00:00.000Z,"[stablecoin, asset-backed-stablecoin, binance-...",NaN,2.051725e+10,2.051725e+10,NaN,6,None,None,None,2022-09-16T00:08:00.000Z,1.000654,1.017964e+10,2.3768,0.090223,0.051758,0.062178,0.057110,0.219356,-0.146026,2.053068e+10,2.1366,2.053068e+10,None,2022-09-16T00:08:00.000Z,1839.0,BNB,BNB,bnb,BUSD-BD1,2022-09-15 20:13:21.320118
6,52,XRP,XRP,xrp,814,2013-08-04T00:00:00.000Z,"[medium-of-exchange, enterprise-solutions, arr...",1.000000e+11,4.982602e+10,9.998930e+10,NaN,7,None,None,None,2022-09-16T00:08:00.000Z,0.326130,1.516843e+09,24.0545,-0.080269,-4.626702,-4.115716,-13.650309,-5.457331,1.453442,1.624976e+10,1.6914,3.261300e+10,None,2022-09-16T00:08:00.000Z,NaN,NaN,NaN,NaN,NaN,2022-09-15 20:13:21.320118
7,2010,Cardano,ADA,cardano,570,2017-10-01T00:00:00.000Z,"[dpos, pos, platform, research, smart-contract...",4.500000e+10,3.418204e+10,3.485495e+10,NaN,8,None,None,None,2022-09-16T00:08:00.000Z,0.465750,6.798961e+08,-7.3005,0.242527,-3.011578,-2.789682,-16.527078,3.576316,-4.640849,1.592030e+10,1.6568,2.095877e+10,None,2022-09-16T00:08:00.000Z,NaN,NaN,NaN,NaN,NaN,2022-09-15 20:13:21.320118
8,5426,Solana,SOL,solana,384,2020-04-10T00:00:00.000Z,"[pos, platform, solana-ecosystem, cms-holdings...",NaN,3.539797e+08,5.116169e+08,

Function to automate the API pull

In [45]:
def api_executable():
    global df  # Declare our first data frame as a global variable

    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
    'start':'1',
    'limit':'10',  # top 10
    'convert':'USD'
    }
    headers = {
    'Accepts': 'application/json',
    'X-CMC_PRO_API_KEY': 'be418cc8-d60e-4d91-85a8-2bfae177f020',
    }

    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        # print(data)  
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)

    df_2 = pd.json_normalize(data['data'])  # create a second data frame to append the data
    df_2["timestamp"] = pd.Timestamp("now")
    df = pd.concat([df_2, df])  # append the data to the first data frame / pd.concat() is the replacement for append()

Creating loop to run the function

In [46]:
for i in range(333):  # 333 credits available for Coinmarketcap API pull
    api_executable()
    print("API Executable has completed successfully")
    sleep(60)  # Sleep for 1 minute
exit()

API Executable has completed successfully
API Executable has completed successfully


KeyboardInterrupt: 

In [47]:
df

,id,name,symbol,slug,num_market_pairs,date_added,tags,max_supply,circulating_supply,total_supply,platform,cmc_rank,self_reported_circulating_supply,self_reported_market_cap,tvl_ratio,last_updated,quote.USD.price,quote.USD.volume_24h,quote.USD.volume_change_24h,quote.USD.percent_change_1h,quote.USD.percent_change_24h,quote.USD.percent_change_7d,quote.USD.percent_change_30d,quote.USD.percent_change_60d,quote.USD.percent_change_90d,quote.USD.market_cap,quote.USD.market_cap_dominance,quote.USD.fully_diluted_market_cap,quote.USD.tvl,quote.USD.last_updated,platform.id,platform.name,platform.symbol,platform.slug,platform.token_address,timestamp
0,1,Bitcoin,BTC,bitcoin,9745,2013-04-28T00:00:00.000Z,"[mineable, pow, sha-256, store-of-value, state...",2.100000e+07,1.915173e+07,1.915173e+07,NaN,1,None,None,None,2022-09-16T00:17:00.000Z,19658.319274,3.632469e+10,-4.0070,-0.128325,-2.921170,1.565908,-17.861064,-5.843446,-4.530157,3.764908e+11,39.2026,4.128247e+11,None,2022-09-16T00:17:00.000Z,NaN,NaN,NaN,NaN,NaN,2022-09-15 20:18:52.164007
1,1027,Ethereum,ETH,ethereum,6091,2015-08-07T00:00:00.000Z,"[mineable, pos, smart-contracts, ethereum-ecos...",NaN,1.223842e+08,1.223842e+08,NaN,2,None,None,None,2022-09-16T00:17:00.000Z,1464.429920,2.685229e+10,49.0936,-0.310071,-10.803197,-10.645104,-22.245803,8.527452,34.205748,1.792231e+11,18.6583,1.792231e+11,None,2022-09-16T00:17:00.000Z,NaN,NaN,NaN,NaN,NaN,2022-09-15 20:18:52.164007
2,825,Tether,USDT,tether,39753,2015-02-25T00:00:00.000Z,"[payments, stablecoin, asset-backed-stablecoin...",NaN,6.792344e+10,7.015678e+10,NaN,3,None,None,None,2022-09-16T00:17:00.000Z,1.000011,6.051425e+10,9.8042,0.002428,-0.014474,-0.007671,-0.008427,0.030727,0.109052,6.792418e+10,7.0727,7.015755e+10,None,2022-09-16T00:17:00.000Z,1027.0,Ethereum,ETH,ethereum,0xdac17f958d2ee523a2206206994597c13d831ec7,2022-09-15 20:18:52.164007
3,3408,USD Coin,USDC,usd-coin,6229,2018-10-08T00:00:00.000Z,"[medium-of-exchange, stablecoin, asset-backed-...",NaN,5.035820e+10,5.035820e+10,NaN,4,None,None,None,2022-09-16T00:17:00.000Z,0.999847,7.299172e+09,26.4229,-0.009835,-0.011399,-0.015182,-0.026533,-0.043373,-0.051289,5.035051e+10,5.2418,5.035051e+10,None,2022-09-16T00:17:00.000Z,1027.0,Ethereum,ETH,ethereum,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,2022-09-15 20:18:52.164007
4,1839,BNB,BNB,bnb,1103,2017-07-25T00:00:00.000Z,"[marketplace, centralized-exchange, payments, ...",2.000000e+08,1.613373e+08,1.613373e+08,NaN,5,None,None,None,2022-09-16T00:17:00.000Z,269.640286,1.081066e+09,13.2589,-0.005913,-3.696457,-4.087125,-14.919970,8.228914,24.177502,4.350303e+10,4.5329,5.392806e+10,None,2022-09-16T00:17:00.000Z,NaN,NaN,NaN,NaN,NaN,2022-09-15 20:18:52.164007
5,4687,Binance USD,BUSD,binance-usd,5095,2019-09-20T00:00:00.000Z,"[stablecoin, asset-backed-stablecoin, binance-...",NaN,2.051725e+10,2.051725e+10,NaN,6,None,None,None,2022-09-16T00:17:00.000Z,0.999817,1.019180e+10,2.7271,-0.026115,-0.047330,-0.017707,-0.019347,0.155885,-0.279341,2.051351e+10,2.1356,2.051351e+10,None,2022-09-16T00:17:00.000Z,1839.0,BNB,BNB,bnb,BUSD-BD1,2022-09-15 20:18:52.164007
6,52,XRP,XRP,xrp,814,2013-08-04T00:00:00.000Z,"[medium-of-exchange, enterprise-solutions, arr...",1.000000e+11,4.982602e+10,9.998930e+10,NaN,7,None,None,None,2022-09-16T00:17:00.000Z,0.326182,1.520039e+09,24.5586,0.178676,-4.623103,-4.323791,-14.030969,-5.708264,0.912874,1.625236e+10,1.6924,3.261821e+10,None,2022-09-16T00:17:00.000Z,NaN,NaN,NaN,NaN,NaN,2022-09-15 20:18:52.164007
7,2010,Cardano,ADA,cardano,570,2017-10-01T00:00:00.000Z,"[dpos, pos, platform, research, smart-contract...",4.500000e+10,3.418204e+10,3.485495e+10,NaN,8,None,None,None,2022-09-16T00:17:00.000Z,0.464973,6.819544e+08,-6.8258,0.115078,-3.228417,-3.591160,-17.011632,3.206648,-5.415247,1.589373e+10,1.6546,2.092379e+10,None,2022-09-16T00:17:00.000Z,NaN,NaN,NaN,NaN,NaN,2022-09-15 20:18:52.164007
8,5426,Solana,SOL,solana,384,2020-04-10T00:00:00.000Z,"[pos, platform, solana-ecosystem, cms-holdings...",NaN,3.539797e+08,5.116169